In [1]:
# Импорт всех нужных библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Загрузка данных
train_data = pd.read_parquet('train_dataset_hackaton2023_train.parquet')

# Преобразование столбца с датой и временем в соответствующий формат
train_data['startdatetime'] = pd.to_datetime(train_data['startdatetime'])

In [3]:
# Извлечение дня недели (где понедельник = 0, воскресенье = 6)
train_data['day_of_week'] = train_data['startdatetime'].dt.dayofweek

In [8]:
# Создаем копию исходных данных с сортировкой для дальнейшей обработки.
sorted_data = train_data.sort_values(['customer_id', 'startdatetime']).copy()

# Удаляем дубликаты по 'customer_id' и 'startdatetime' для получения уникальных дат покупок.
unique_purchase_dates = sorted_data.drop_duplicates(subset=['customer_id', 'startdatetime']).copy()

# Вычисляем интервалы между покупками.
purchase_intervals = unique_purchase_dates.groupby('customer_id')['startdatetime'].diff().dt.days

# Присваиваем вычисленные интервалы в DataFrame.
unique_purchase_dates.loc[:, 'purchase_interval'] = purchase_intervals

# Удаляем NaN значения, появившиеся в результате использования функции diff().
intervals_data = unique_purchase_dates.dropna(subset=['purchase_interval'])

# Теперь удаляем интервалы, равные нулю.
intervals_data = intervals_data[intervals_data['purchase_interval'] != 0]

# Группировка по 'customer_id' и подсчет количества уникальных и общего числа интервалов между покупками.
intervals_analysis = intervals_data.groupby('customer_id')['purchase_interval'].agg(['nunique', 'count'])

# Выбираем клиентов, у которых более двух покупок и одинаковый интервал между покупками.
regular_customers = intervals_analysis[(intervals_analysis['nunique'] == 1) & (intervals_analysis['count'] > 2)]

# Получаем список ID клиентов.
regular_customer_ids = regular_customers.index.tolist()

# Количество таких клиентов.
regular_customer_count = len(regular_customer_ids)

print(f"Количество клиентов с одинаковыми интервалами между более чем двумя покупками: {regular_customer_count}")
print("ID этих клиентов:", regular_customer_ids)


Количество клиентов с одинаковыми интервалами между более чем двумя покупками: 364
ID этих клиентов: [274671, 287550, 419107, 650516, 1483236, 1516020, 1764017, 1830984, 1836809, 1934954, 2070964, 2092991, 2100080, 2145081, 2635398, 2693983, 2756650, 2785066, 2854497, 2938927, 3139573, 3190465, 3827374, 3938194, 4007394, 4093524, 4109974, 4139426, 4621472, 5226306, 5325604, 5658661, 5679329, 5891878, 5949030, 6155659, 6179198, 6206655, 6386089, 6400261, 6513144, 6583065, 6675888, 6686782, 6794406, 6866560, 7071581, 7091479, 7295027, 7316823, 7355240, 7773336, 7776484, 7782210, 8011337, 8209749, 8259996, 8366441, 8439021, 8484098, 8696178, 8828861, 9125515, 9683925, 9731866, 9739350, 10070566, 10164119, 10263719, 10784008, 10893172, 10909939, 10999871, 11059171, 11124428, 11164138, 11353496, 11456790, 11460025, 11551678, 11570944, 11581679, 11612037, 11669243, 11743961, 11912719, 12011652, 12030806, 12045447, 12069034, 12081242, 12097852, 12362038, 12403486, 12404475, 12407663, 12699071

In [9]:
# Отфильтровываем исходник DataFrame для получения только тех клиентов, которые у нас есть в списке
regular_customers_df = sorted_data[sorted_data['customer_id'].isin(regular_customer_ids)]

# Группируем отфильтрованные данные по 'customer_id' и 'buy_post', чтобы узнать, сколько покупок каждого класса сделал каждый клиент
class_counts = regular_customers_df.groupby(['customer_id', 'buy_post']).size().unstack(fill_value=0)

# Теперь у нас есть DataFrame, где индексы - это customer_id, а столбцы - это значения 'buy_post'
# Считаем количество клиентов для каждого класса
class_distribution = class_counts.sum(axis=0)

# Получаем количество уникальных клиентов для каждого класса 'buy_post'
unique_customer_count_by_class = regular_customers_df.drop_duplicates(subset=['customer_id', 'buy_post'])['buy_post'].value_counts()

print(f"Количество уникальных регулярных клиентов с 'buy_post' равным 1: {unique_customer_count_by_class.get(1, 0)}")
print(f"Количество уникальных регулярных клиентов с 'buy_post' равным 0: {unique_customer_count_by_class.get(0, 0)}")


Количество уникальных регулярных клиентов с 'buy_post' равным 1: 213
Количество уникальных регулярных клиентов с 'buy_post' равным 0: 151


In [10]:
# Выбираем данные для пользователя с ID 274671
customer_purchases = sorted_data[sorted_data['customer_id'] == 274671]

# Рассчитываем интервал между покупками
customer_purchases['purchase_interval'] = customer_purchases['startdatetime'].diff().dt.days

# Удаляем NaN значения, которые появляются для первой покупки, так как для неё не существует предыдущей
customer_intervals = customer_purchases['purchase_interval'].dropna()

print(f"Интервалы между покупками пользователя 274671: {customer_intervals.tolist()}")

Интервалы между покупками пользователя 274671: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


/var/folders/ng/zvvj96k93v5f0p56hzn85y_m0000gp/T/ipykernel_3111/419282486.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_purchases['purchase_interval'] = customer_purchases['startdatetime'].diff().dt.days
